In [102]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup 
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
import pymongo


In [103]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.mars_db
collection = db.marstitle

In [104]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [105]:
# Visit the mars news site
url = 'https://redplanetscience.com/'
browser.visit(url)

In [106]:
# HTML Parser
response = requests.get(url)
news_soup = BeautifulSoup(response.text, 'html.parser')
slide_elems = news_soup.find_all('div', class_='content_title')

In [107]:
# loop over results to get article data
for slide_elem in slide_elems:
    # scrape the article title 
    news_title = slide_elem.find('div', class_='content_title').get_text()
    
    # scrape the article preview
    news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
        
    # print article data
    print('-----------------')
    print(news_title)
    print(news_p)

    # Dictionary to be inserted into MongoDB
    post = {
        'title': news_title,
        'preview': news_p,
    }

    # Insert dictionary into MongoDB as a document
    collection.insert_one(post)

In [110]:
# Display the MongoDB records created above
marstitles = db.marstitle.find()
for marstitle in marstitles:
    print(marstitle)